In [1]:
import os
import json
import pandas as pd
import numpy as np

current_directory = os.getcwd()
path_components = current_directory.split(os.path.sep)
results_dir = os.path.sep.join(path_components[:-3])


In [2]:
def find_largest_iteration(folder_path):
    subdirectories = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    iteration_numbers = [int(d.split('_')[1]) for d in subdirectories if d.startswith('iter_')]
    if not iteration_numbers:
        return None
    largest_iteration_folder = os.path.join(folder_path, f'iter_{max(iteration_numbers)}')
    return largest_iteration_folder

def find_loss_csv(folder_path):
    largest_iteration_folder = find_largest_iteration(os.path.join(folder_path,'iterations'))
    if largest_iteration_folder:
        loss_csv_path = os.path.join(largest_iteration_folder, 'loss.csv')
        if os.path.isfile(loss_csv_path):
            return loss_csv_path
    return None

def get_last_iteration_losses(loss_csv_path):
    df = pd.read_csv(loss_csv_path)
    last_row = df.iloc[-1]
    losses = last_row[list(df.columns)]
    return losses

def read_results_json(folder_path):
    results_json_path = os.path.join(folder_path, 'results_values.json')
    if os.path.isfile(results_json_path):
        with open(results_json_path, 'r') as json_file:
            results_data = json.load(json_file)
        return results_data
    else:
        return None

def get_results_by_sim(folder_path):
    loss_csv_path = find_loss_csv(folder_path)
    if loss_csv_path:
        results = get_last_iteration_losses(loss_csv_path)
        results_data = read_results_json(folder_path)
        if results_data:
            for name in results_data:
                results[name] = float(results_data[name])
            return results
    return None

In [3]:
molecule = 'methanol'
results_dir = os.path.join(results_dir,'.results',molecule)
print(results_dir)

/home/martinachondo/XPBE/.results/methanol


In [4]:
folders = ['']
data = dict()
for folder in folders:
    folder_path = os.path.join(results_dir,folder) if folder!= '' else results_dir
    sims = os.listdir(folder_path)
    for sim in sims:
        sim_path = os.path.join(folder_path,sim)
        result = get_results_by_sim(sim_path)
        data[sim] = result

data_filtered = {sim: result for sim, result in data.items() if result is not None}
df = pd.DataFrame.from_dict(data_filtered, orient='index')

filter_list = ["Gsolv_value",
                "Loss_XPINN",
                "Loss_NN1",
                "Loss_NN2",
                "Loss_Val_NN1",
                "Loss_Val_NN2",
                "Loss_continuity_u",
                "Loss_continuity_du",
                "Loss_Residual_R1",
                "Loss_Residual_R2",
                "Loss_Boundary_D2",
                "Loss_Data_K2",
                "L2_analytic"]
df_filtered = df.filter(filter_list)

excel_file_path = 'results_temp.xlsx'
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='DF_complete', index=True)
    df_filtered.to_excel(writer, sheet_name='DF_Filtered', index=True)

In [5]:
df_filtered

,Gsolv_value,Loss_XPINN,Loss_NN1,Loss_NN2,Loss_Val_NN1,Loss_Val_NN2,Loss_continuity_u,Loss_continuity_du,Loss_Residual_R1,Loss_Residual_R2,Loss_Boundary_D2,Loss_Data_K2
M2,-8.505,0.000318,0.000173,0.000173,0.000172,0.000172,5.308000e-06,1.678000e-04,1.495000e-10,8.939000e-13,3.072000e-08,3.502000e-08
M4,-11.580,0.000660,0.000058,0.000058,0.000060,0.000063,1.430000e-08,5.712000e-05,9.752000e-07,4.023000e-07,3.662000e-09,3.484000e-09
M1,-11.643,0.000160,0.000008,0.000007,0.000015,0.000010,3.045000e-08,6.966000e-06,1.204000e-06,1.557000e-07,3.203000e-09,2.522000e-09
M5,-11.909,0.000604,0.000287,0.000287,0.000304,0.000304,8.010000e-08,2.871000e-04,1.895000e-09,2.046000e-13,5.338000e-09,5.795000e-09
M6,-11.580,0.000118,0.000010,0.000009,0.000010,0.000008,1.946000e-08,8.663000e-06,1.297000e-06,8.698000e-08,2.515000e-09,2.369000e-09
M3,-12.080,0.000027,0.000001,0.000001,0.000004,0.000003,1.643000e-07,3.590000e-07,5.751000e-07,5.859000e-07,8.318000e-08,8.353000e-08
